In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
df=pd.read_csv(r"C:\Users\prajw\Data Science\Assignments\15 Random forests\Fraud_check.csv")

In [3]:
df.head(10)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
5,NO,Divorced,33329,116382,0,NO
6,NO,Divorced,83357,80890,8,YES
7,YES,Single,62774,131253,3,YES
8,NO,Single,83519,102481,12,YES
9,YES,Divorced,98152,155482,4,YES


In [4]:
df.describe()

,Taxable.Income,City.Population,Work.Experience
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


In [5]:
df.skew(),df.kurt()

C:\Users\prajw\AppData\Local\Temp\ipykernel_29432\278608664.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.skew(),df.kurt()


(Taxable.Income     0.030015
 City.Population    0.125009
 Work.Experience    0.018529
 dtype: float64,
 Taxable.Income    -1.199782
 City.Population   -1.120154
 Work.Experience   -1.167524
 dtype: float64)

In [6]:
df.corr()

,Taxable.Income,City.Population,Work.Experience
Taxable.Income,1.000000,-0.064387,-0.001818
City.Population,-0.064387,1.000000,0.013135
Work.Experience,-0.001818,0.013135,1.000000


In [7]:
df1=pd.get_dummies(df,columns=["Undergrad","Urban","Marital.Status"])

In [8]:
df1["Taxable_Income"]="Good"
df1.loc[df1["Taxable.Income"]<=30000,"Taxable_Income"]="Risky"
df1=df1.drop(columns="Taxable.Income")

In [9]:
df1["Taxable_Income"].unique()

array(['Good', 'Risky'], dtype=object)

In [10]:
x=df1.iloc[:,:-1]
y=df1.iloc[:,-1]

num_trees=200
max_features=6

kfold=KFold(n_splits=8,shuffle=True,random_state=6)

model=RandomForestClassifier(n_estimators=num_trees,max_features=max_features)
result=cross_val_score(model,x,y,cv=kfold)
result.mean()

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20)

In [12]:
model=RandomForestClassifier(n_jobs=4,n_estimators=150,oob_score=True,criterion="entropy")

In [13]:
model.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=150, n_jobs=4,
                       oob_score=True)

In [14]:
model.oob_score_

0.7479166666666667

In [15]:
pred_train=model.predict(x_train)

In [16]:
accuracy_score(y_train,pred_train)

1.0

In [17]:
confusion_matrix(y_train,pred_train)

array([[380,   0],
       [  0, 100]], dtype=int64)

In [18]:
pred_test=model.predict(x_test)

In [19]:
accuracy_score(y_test,pred_test)

0.6916666666666667

In [20]:
confusion_matrix(y_test,pred_test)

array([[82, 14],
       [23,  1]], dtype=int64)

In [21]:
df_t=pd.DataFrame({"Actual":y_test,"Predicted":pred_test})

In [22]:
df_t

,Actual,Predicted
523,Good,Good
381,Good,Good
584,Good,Good
157,Good,Good
102,Good,Good
...,...,...
544,Risky,Good
88,Good,Risky
572,Good,Good
516,Good,Risky
